In [2]:
import pandas as pd 
from sqlalchemy import create_engine ,Integer,String,Date,VARCHAR,text

In [3]:

pd.set_option('display.max_columns', None)
caminho_do_arquivo = r"C:\curso-eng-dados\Python-SGBDS\Arquivos/Origem de dados/V_OCORRENCIA_AMPLA.json"
df = pd.read_json(caminho_do_arquivo, encoding='utf-8-sig')
colunas = ["Numero_da_Ocorrencia", "Classificacao_da_Ocorrência", "Data_da_Ocorrencia","Municipio","UF","Regiao","Nome_do_Fabricante","Modelo"]
df = df[colunas]
df.rename( columns={  'Classificacao_da_Ocorrência' : 'Classificacao_da_Ocorrencia'  } ,inplace=True )

In [5]:
df.head()

,Numero_da_Ocorrencia,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Municipio,UF,Regiao,Nome_do_Fabricante,Modelo
0,7762,Incidente,2018-03-21,SÃO PAULO,SP,Sudeste,AGUSTA,AW109SP
1,7759,Acidente,2018-03-14,MONTES CLAROS,MG,Sudeste,CESSNA AIRCRAFT,A152
2,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT,T188C
3,7758,Acidente,2018-01-26,INACIOLÂNDIA,GO,Centro-Oeste,CESSNA AIRCRAFT,T188C
4,7757,Incidente Grave,2018-03-18,TORRES,RS,Sul,PIPER AIRCRAFT,PA-34-200


In [7]:
df.shape

(13011, 8)

In [9]:
from datetime import datetime,timedelta

#Metodo 1 
data_atual = datetime.now()#data atual
data_ha_3_meses = data_atual - timedelta(days=3*30)# subtrai 3 meses usando time delta
data_ha_3_meses_apenas_data = data_ha_3_meses.date() #traz somente a data, exclui hora minuto e etc

#obs não levar em consideração variação nos numeros de dias em cada mes
print(data_ha_3_meses_apenas_data)

2025-01-26


In [12]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

#metodo 2 
data_atual2 = datetime.now()
d_menos3M = data_atual - relativedelta(month=3) #subtrai 3 meses usando relativedelta
dt_d_menos3M = d_menos3M.date()

#obs> leva em consideração a variação nos numeros de dias em cada mes, forncendo um resultado mais preciso
print(dt_d_menos3M)


2025-03-26


In [13]:
df.dtypes

Numero_da_Ocorrencia                    int64
Classificacao_da_Ocorrencia            object
Data_da_Ocorrencia             datetime64[ns]
Municipio                              object
UF                                     object
Regiao                                 object
Nome_do_Fabricante                     object
Modelo                                 object
dtype: object

In [ ]:
# converter tudo para pandas to_dateime para fazer o filtro na base
df['Data_da_Ocorrencia'] = pd.to_datetime(df['Data_da_Ocorrencia'])
data_ha_3_meses_apenas_data = pd.to_datetime(data_ha_3_meses_apenas_data)
dt_d_menos3M = pd.to_datetime(dt_d_menos3M)

In [14]:
df = df[df['Data_da_Ocorrencia'] >= dt_d_menos3M]

TypeError: Invalid comparison between dtype=datetime64[ns] and date

In [ ]:
df.head()

In [ ]:
dt_d_menos3M_ajustado = dt_d_menos3M.date()

In [ ]:
dbname   = 'python'
user     = 'postgres'
host     = 'localhost'
password = '******'
port     = '5432' 

conexao_str = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(conexao_str)

nome_tabela = 'anac_sqlalchemy'

#Deletar registros com base no ano atual
cursor=engine.connect()
delete = text(f''' delete from public.{nome_tabela}	
                   WHERE "Data_da_Ocorrencia" >= '{dt_d_menos3M_ajustado}'
                ''')
cursor.execute(delete)
cursor.commit()

# Enviar DataFrama para o BD
df.to_sql(nome_tabela,engine,index=False, if_exists='replace',dtype={

                'Numero_da_Ocorrencia' : Integer ,
                'Classificacao_da_Ocorrencia': VARCHAR(50),
                'Data_da_Ocorrencia':Date

})
engine.dispose()
cursor.close()

ProgrammingError: (psycopg2.errors.UndefinedTable) ERRO:  relação "puclib.anac_sqlalchemy" não existe
LINE 1: DELETE FROM puclib.anac_sqlalchemy WHERE EXTRACT(YEAR FROM "...
                    ^

[SQL: DELETE FROM puclib.anac_sqlalchemy WHERE EXTRACT(YEAR FROM "Data_da_Ocorrencia") = 2025]
(Background on this error at: https://sqlalche.me/e/20/f405)